# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#------------------------------------------------------------------------
#-------------------------Imports----------------------------------------
#------------------------------------------------------------------------
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from torch import nn
import pickle

from pandas import DataFrame
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy import stats
import os
import time
import random
import seaborn as sns
import time
import datetime
import math
import csv
import networkx as nx

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.model_selection import GridSearchCV
from torch.autograd import Variable


# Variables

In [ ]:
#------------------------------------------------------------------------
#-------------------------Variables--------------------------------------
#------------------------------------------------------------------------
data_name = 'la'  # 'la'--> METR-LA, 'bay'-->PEMS-BAY
model_name =  'stat_pytorch_15'  #'stat_pytorch_15' 'transformer_pytorch_11'
train_rate, val_rate = 0.7, 0.1  # 70%(train data), 10% (validation data) and 20%(test data)
interval = 200 # for displaying results
seq_len = 12  # Inputs sequence length
num_runs = 1  # Test number

pre_len = 3  ## Outputs sequence length

if (data_name == 'la'):
    adj_path =  '/content/drive/My Drive/Congestion/data/adj_mx.pkl' # path of adjacency matrix
    start_date = '2012-03-01 00:00:00'  # Start of training data
    test_date = '2012-06-27 23:55:00'   # Start of testing data
    freq = '5min'  # Frequence

    d_model = 207       # Model dimension 207
    n_heads = 3         # Head number 3
    FFN_units = 64      # Unit number 64
    hid = [128, 64, 16] # Unit number of hidden layer [128, 64, 16]
    n_layers_att = 1    # Layer number of attention 1
    n_layers = 2        # Layer number 2
    dropout = 0.6      # dropout 0.25

    lr = 0.00125        # Learning rate 0.00125
    decay_rate = 0.925  # decay rate 0.925
    n_epochs = 100      # Epochs number 100
    patience = 10       # patience 15
    batch_size = 16     # batch size 16

elif (data_name == 'bay'):
    adj_path =  '/content/drive/My Drive/Congestion/data/adj_mx_bay.pkl' # path of adjacency matrix
    start_date = '2017-01-01 00:00:00'    # Start of training data
    test_date = '2017-06-30 22:55:00'     # Start of testing data
    freq = '5min'   # Frequence

    d_model = 325         # Model dimension 325
    n_heads = 5           # Head number 5
    FFN_units = 64        # Unit number 64
    hid = [512, 128, 64]  # Unit number of hidden layer [512, 128, 64]
    n_layers_att = 1      # Layer number of attention 1
    n_layers = 2          # Layer number 2
    dropout = 0.25        # dropout 0.25

    lr = 0.00125          # Learning rate 0.00125
    decay_rate = 0.95     # decay rate 0.95
    n_epochs = 100        # Epochs number 100
    patience = 15         # patience 15
    batch_size = 64       # batch size 64

else: print("Error.....choose another dataset")


# Functions

In [ ]:
#------------------------------------------------------------------------
#-------------------------Functions--------------------------------------
#------------------------------------------------------------------------

#------------------------------------------------------------------------
#------------------------------------------------------------------------
#-----------------------------make_dir-----------------------------------
#------------------------------------------------------------------------
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
        print("---  new folder  ---", path)
    else:
        print("---  There is this folder!  ---", path)


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#-------------------------get_data---------------------------------------
#------------------------------------------------------------------------
def get_data(seq_len, pre_len, data_name):
    data = load_data(data_name)
    time_len, d_model = data.shape
    rng = pd.date_range(start_date, periods=time_len, freq=freq)
    data.index = rng

    #----------split data----------------------------
    trainX, trainY, valX, valY, testX, testY = split_data(data, train_rate, val_rate, seq_len, pre_len)

    return trainX, trainY, valX, valY, testX, testY


#------------------------------------------------------------------------
#-------------------------load data--------------------------------------
#------------------------------------------------------------------------
def load_data(data_name=data_name):
    if (data_name == 'la'):
        data_path = r'/content/drive/My Drive/Congestion/metr_la.csv'
    elif (data_name == 'bay'):
        data_path = r'/content/drive/My Drive/Congestion/pems_bay.csv'
    else:
        return print("Error.....choose another dataset")

    data = pd.read_csv(data_path)
    data.drop(columns=data.columns[0], axis=1, inplace=True)

    return data

#------------------------------------------------------------------------
#-------------------------split_data-------------------------------------
#------------------------------------------------------------------------
def split_data(data, train_rate, val_rate, seq_len, pre_len):

    dataX, dataY = [], []
    for i in range(len(data) - (seq_len + pre_len)):
        a = data[i: i + seq_len + pre_len]
        dataX.append(a[0 : seq_len])
        dataY.append(a[seq_len : seq_len + pre_len])

    dataX, dataY = np.array(dataX), np.array(dataY)
    data_size = dataX.shape[0]
    train_size = int(data_size * train_rate)
    val_size = int(data_size * val_rate)

    trainX, trainY = dataX[0:train_size], dataY[0:train_size]
    valX, valY = dataX[train_size:train_size + val_size], dataY[train_size:train_size + val_size]
    testX, testY = dataX[train_size + val_size:data_size], dataY[train_size + val_size:data_size]

    return trainX, trainY, valX, valY, testX, testY


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#---------------------------load_dataset---------------------------------
#------------------------------------------------------------------------
def load_dataset(device, trainX, trainY, valX, valY, testX, testY, batch_size):
    data = {}
    scaler = StandardScaler(mean=trainX.mean(), std=trainX.std())
    trainX = scaler.transform(trainX)
    valX = scaler.transform(valX)
    testX = scaler.transform(testX)

    data['train_loader'] = Data_Loader(trainX, trainY, batch_size)
    data['val_loader'] = Data_Loader(valX, valY, batch_size)
    data['test_loader'] = Data_Loader(testX, testY, batch_size)
    data['scaler'] = scaler
    return data


#------------------------------------------------------------------------
#-------------------------StandardScaler---------------------------------
#------------------------------------------------------------------------
class StandardScaler():

    def __init__(self, mean, std, fill_zeroes=True):
        self.mean = mean
        self.std = std
        self.fill_zeroes = fill_zeroes

    def transform(self, data):
        if self.fill_zeroes:
            mask = (data == 0)
            data[mask] = self.mean
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean


#------------------------------------------------------------------------
#-------------------------Data_Loader------------------------------------
#------------------------------------------------------------------------
class Data_Loader(object):
    def __init__(self, xs, ys, batch_size):

        self.batch_size = batch_size
        self.current_ind = 0
        self.size = len(xs)
        self.num_batch = int(self.size // self.batch_size)
        self.xs = xs
        self.ys = ys

    def shuffle(self):
        permutation = np.random.permutation(self.size)
        xs, ys = self.xs[permutation], self.ys[permutation]
        self.xs = xs
        self.ys = ys

    def get_iterator(self):
        self.current_ind = 0

        def _wrapper():
            while self.current_ind < self.num_batch:
                start_ind = self.batch_size * self.current_ind
                end_ind = min(self.size, self.batch_size * (self.current_ind + 1))
                x_i = self.xs[start_ind: end_ind, ...]
                y_i = self.ys[start_ind: end_ind, ...]
                yield (x_i, y_i)
                self.current_ind += 1

        return _wrapper()


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#-----------------load_pickle (Adjacency matrix)-------------------------
#------------------------------------------------------------------------
def load_pickle(pickle_file):
    try:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
    except UnicodeDecodeError as e:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
    except Exception as e:
        print('Unable to load data ', pickle_file, ':', e)
        raise
    return pickle_data


#------------------------------------------------------------------------
#--------------------Adjacency matrix normalization----------------------
#------------------------------------------------------------------------
def get_normalized_adj(A):

    I = np.eye(A.shape[0], dtype=np.float32)
    A_hat = A + I # add self-loops
    D_hat_diag = np.sum(A_hat, axis=1)
    D_hat_diag_inv_sqrt = np.power(D_hat_diag, -0.5)
    D_hat_diag_inv_sqrt[np.isinf(D_hat_diag_inv_sqrt)] = 0.
    D_hat_inv_sqrt = np.diag(D_hat_diag_inv_sqrt)
    return np.dot(np.dot(D_hat_inv_sqrt, A_hat), D_hat_inv_sqrt)


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#-------------------------learning rate----------------------------------
#------------------------------------------------------------------------
def learning_r(epoch, decay_rate):
    return decay_rate**epoch


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#-------------------------evaluation-------------------------------------
#------------------------------------------------------------------------
def evaluate(pred, target):
    mape = loss_mape(pred, target, 0.0).item()
    rmse = loss_rmse(pred, target, 0.0).item()
    mae = loss_mae(pred, target, 0.0).item()
    return mape, rmse, mae


#------------------------------------------------------------------------
#-------------------------loss_mae---------------------------------------
#------------------------------------------------------------------------
def loss_mae(preds, labels, null_val=np.nan):
#    print('preds', preds.shape)
#    print('labels', labels.shape)
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels != null_val)
    mask = mask.float()
    mask /= torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = torch.abs(preds - labels)
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)


#------------------------------------------------------------------------
#-------------------------loss_rmse--------------------------------------
#------------------------------------------------------------------------
def loss_rmse(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels != null_val)
    mask = mask.float()
    mask /= torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = (preds - labels)** 2
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.sqrt(torch.mean(loss))


#------------------------------------------------------------------------
#-------------------------loss_mape--------------------------------------
#------------------------------------------------------------------------
def loss_mape(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels != null_val)
    # print("mask:",type(mask))
    mask = mask.float()
    mask /= torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = torch.abs((preds - labels) / labels)
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)*100


# Positional Encoding

In [ ]:
#------------------------------------------------------------------------
#-------------------------PositionalEncoding-----------------------------
#------------------------------------------------------------------------
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len, device):
        super(PositionalEncoding, self).__init__()

    def get_angles(self, pos, i, d_model): # pos: (seq_length, 1) i: (1, d_model)
        angles = 1 / np.power(10000., (2*(i//2)) / np.float32(d_model))
        return pos * angles # (seq_length, d_model)

    def forward(self, inputs):
        # input shape batch_size, seq_length, d_model
        seq_length = inputs.size(-2)
        d_model = inputs.size(-1)
        # Calculate the angles given the input
        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],
                                 np.arange(d_model)[np.newaxis, :],
                                 d_model)
        # Calculate the positional encodings
        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        # Expand the encodings with a new dimension
        pos_encoding = angles[np.newaxis, ...]
        pos_encoding = torch.from_numpy(pos_encoding).to(torch.float32).to(device)
#        pos_encoding = torch.from_numpy(pos_encoding).to(torch.float32)

        return inputs + pos_encoding


# GCN

In [ ]:
#------------------------------------------------------------------------
#-------------------------------GCN--------------------------------------
#------------------------------------------------------------------------
class GCN(nn.Module):
    def __init__(self, node_features, hidden_dim, num_classes, dropout, n_layers, use_bias=True):
        super(GCN, self).__init__()
        self.n_layers = n_layers
        self.gcn_1 = GCNLayer(node_features, hidden_dim, use_bias)
        self.gcn_2 = GCNLayer(hidden_dim, num_classes, use_bias)
        self.dropout = nn.Dropout(dropout)

    def initialize_weights(self):
        self.gcn.initialize_weights()
        self.gcn_1.initialize_weights()
        self.gcn_2.initialize_weights()

    def forward(self, x, adj):
        x = x.permute(1, 0, 2)
        x = F.relu(self.gcn_1(x, adj))
        x = self.dropout(x)
        x = self.gcn_2(x, adj)
        x = x.permute(1, 2, 0)

        return x


#------------------------------------------------------------------------
#-------------------------------GCNLayer---------------------------------
#------------------------------------------------------------------------
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features, use_bias=True):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(torch.zeros(size=(in_features, out_features))))
        if use_bias:
            self.bias = nn.Parameter(torch.FloatTensor(torch.zeros(size=(out_features,))))
        else:
            self.register_parameter('bias', None)
        self.in_features = in_features
        self.out_features = out_features

        self.initialize_weights()

    def initialize_weights(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        d_model, batch, seq = x.size()
        x = torch.reshape(x, (d_model, -1))
        x = torch.spmm(adj, x)
        x = torch.reshape(x, (d_model, batch, -1))
        x = torch.reshape(x, (d_model * batch, -1))
        x = x @ self.weight
        if self.bias is not None:
            x += self.bias
        x = torch.reshape(x, (-1, batch*self.out_features))
        x = torch.reshape(x, (-1, batch, self.out_features))

        return x


# GRU

In [ ]:
#------------------------------------------------------------------------
#-------------------------------GRU--------------------------------------
#------------------------------------------------------------------------
class GRU(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes, num_layers):
        super(GRU, self).__init__()

        self.gru = nn.GRU(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):

        x, (h_n, h_c) = self.gru(x, None)
        x = self.fc(x)

        return x


# Attention

In [ ]:
#------------------------------------------------------------------------
#-------------------------------Attention--------------------------------
#------------------------------------------------------------------------
class Attention(nn.Module):
    def __init__(self, D, H, FFN_units, dropout_rate):
        super(Attention, self).__init__()

        self.mha1 = MultiHeadAttention(D, H)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.layernorm1 = nn.LayerNorm(D, eps=1e-12) # 1e-9/1e-12

        self.mlp_hidden = nn.Linear(D, FFN_units)
        self.mlp_out = nn.Linear(FFN_units, D)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.layernorm2 = nn.LayerNorm(D, eps=1e-12) # 1e-9/1e-12

    def forward(self, dec, enc, mask1):

        attn1, attn_weights1 = self.mha1(q=dec, k=enc, v=enc, mask=mask1)  # (B, S, D)
        attn1 = self.dropout2(attn1) # (B,S,D)
        attn1 = self.layernorm2(attn1 + dec) # (B,S,D)

        mlp_act = F.relu(self.mlp_hidden(attn1))
        mlp_act = self.mlp_out(mlp_act)
        mlp_act = self.dropout2(mlp_act)
        output = self.layernorm2(mlp_act + attn1)  # (B, S, D)

        return output, attn_weights1


#------------------------------------------------------------------------
#--------------------------MultiHeadAttention----------------------------
#------------------------------------------------------------------------
class MultiHeadAttention(nn.Module):
    '''Multi-head self-attention module'''
    def __init__(self, D, H):
        super(MultiHeadAttention, self).__init__()
        self.H = H # number of heads
        self.D = D # model dimension

        self.wq = nn.Linear(D, D)
        self.wk = nn.Linear(D, D)
        self.wv = nn.Linear(D, D)

        self.dense = nn.Linear(D, D)

    def split_heads(self, tensor):

        batch_size, length, d_model = tensor.size()

        d_tensor = d_model // self.H
        tensor = tensor.view(batch_size, length, self.H, d_tensor).transpose(1, 2)

        return tensor

    def concat_heads(self, tensor):

        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor

    def forward(self, q, k, v, mask):

        q = self.wq(q)  # (B, S, d*H)
        k = self.wk(k)  # (B, S, d*H)
        v = self.wv(v)  # (B, S, d*H)

        q = self.split_heads(q)  # (B, H, S, d)
        k = self.split_heads(k)  # (B, H, S, d)
        v = self.split_heads(v)  # (B, H, S, d)

        attention_scores = torch.matmul(q, k.transpose(-1, -2)) #(B,H,S,S)
        attention_scores = attention_scores / math.sqrt(self.D // self.H)

        # add the mask to the scaled tensor.
        if mask is not None:
            attention_scores += (mask * -1e9)

        attention_weights = nn.Softmax(dim=-1)(attention_scores)
        scaled_attention = torch.matmul(attention_weights, v)  # (B, H, S, d)
        concat_attention = self.concat_heads(scaled_attention) # (B, S, d*H)
        output = self.dense(concat_attention)  # (B, S, D)
        return output, attention_weights


# STAT_model

In [ ]:
#------------------------------------------------------------------------
#-------------------------------STAT_model-------------------------------
#------------------------------------------------------------------------
class STAT_model(nn.Module):

    def __init__(self, device, d_model, n_layers_att, n_layers, FFN_units, hid, dropout_rate):
        super(STAT_model, self).__init__()

        self.device = device
        self.n_layers_att = n_layers_att

        self.adj = self.get_adjacency(adj_path, device)
        self.gcn = GCN(seq_len, FFN_units, seq_len, dropout_rate, n_layers)

        self.pos_encoding = PositionalEncoding(d_model, seq_len, device)
        self.attention = nn.ModuleList([Attention(d_model,
                                                   n_heads,
                                                   FFN_units,
                                                   dropout_rate
                                       ) for i in range(n_layers_att)])

        self.gru = GRU(seq_len, hid[0], hid[1], n_layers)
        self.output1 = nn.Linear(hid[1], hid[2])
        self.output2 = nn.Linear(hid[2], pre_len)


    def forward(self, x):

        x_ = x.permute(0, 2, 1)
        x = self.gcn(x, self.adj)

        x_ = self.pos_encoding(x_)

        mask1 = None
        for i in range(self.n_layers_att):
            x, block = self.attention[i](x_, x, mask1)

        x = x.permute(0, 2, 1)

        x = F.relu(self.gru(x))

        x = self.output1(x)
        x = self.output2(x)

        return x

    def get_adjacency(self, adj_path, device):
        _, _, A = load_pickle(adj_path)
        adj = get_normalized_adj(A)
        adj = torch.from_numpy(adj).to(device)
        return adj


# Main

In [ ]:
#------------------------------------------------------------------------
#------------------------------Main--------------------------------------
#------------------------------------------------------------------------
train_continue = False  # True, False
check_point_exist = False  # True, False

for run in range(num_runs): # num_runs : Test number

    record = []

    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        print("Let's use {} GPU!".format(device))
    else:
        device = torch.device("cpu")

    save = '/content/drive/My Drive/Congestion/JA_model_pytorch/%s/%s/epoch%r'%(model_name,data_name,n_epochs)
    mkdir(save)
    path_1 = 'seq%r_pre%r_run%r_'%(seq_len, pre_len, run+1)+'check_point_model.pt'
    check_point_path = os.path.join(save,path_1)
    record_path = f'{save}/seq%r_pre%r_run%r_'%(seq_len, pre_len, run+1)+'record.csv'

    #----------------------------------------------------------------------
    #---------------------------Preparing data-----------------------------
    #----------------------------------------------------------------------
    trainX, trainY, valX, valY, testX, testY = get_data(seq_len, pre_len, data_name)
    d_model = trainX.shape[-1]
    dataloader = load_dataset(device, trainX, trainY, valX, valY, testX, testY, batch_size)
    scaler = dataloader['scaler']

    #----------------------------------------------------------------------
    #---------------------------Defining model-----------------------------
    #----------------------------------------------------------------------
    model = STAT_model(device = device,
                        d_model = d_model,
                        n_layers_att = n_layers_att,
                        n_layers = n_layers,
                        FFN_units = FFN_units,
                        hid = hid,
                        dropout_rate = dropout
                        ).to(device)
    model.to(device)
    model.zero_grad()
    # Loss and optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    optimizer.zero_grad()
    scheduler = torch.optim.lr_scheduler.LambdaLR(
            optimizer, lr_lambda=lambda epoch: learning_r(epoch, decay_rate))
    criterion = loss_mae

    #----------------------------------------------------------------------
    #----------------Loading model with the last parameters----------------
    #----------------------------------------------------------------------
    if train_continue:
        print("reload the model from :{}", check_point_path)
        checkpoint = torch.load(check_point_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


    print('\n====================Run N°: {}================='.format(run+1))
    his_loss, val_time, train_time = [], [], []
    trigger_times = 0

    for epoch in range(n_epochs):
        print('-' * 10)
        print('Epoch {}/{}'.format(epoch+1, n_epochs))

        #----------------------------------------------------------------------
        #------------------------------Training--------------------------------
        #----------------------------------------------------------------------
        train_loss, train_mape, train_rmse, train_mae = [], [], [], []
        t1, t = time.time(), time.time()
        dataloader['train_loader'].shuffle()
        for iter, (x, y) in enumerate(dataloader['train_loader'].get_iterator()):
            trainX = torch.Tensor(x).to(device).transpose(1, 2)
            trainY = torch.Tensor(y).to(device).transpose(1, 2)

            model.train()
            y_pred = model(trainX)
            y_pred = scaler.inverse_transform(y_pred)
            loss = criterion(y_pred, trainY, 0.0)

            # Backward pass and update
            loss.backward()
            optimizer.step()

            # zero grad before new step
            optimizer.zero_grad()

            evaluation = evaluate(y_pred, trainY)
            train_loss.append(loss.item())
            train_mape.append(evaluation[0])
            train_rmse.append(evaluation[1])
            train_mae.append(evaluation[2])

            if iter % interval == 0:
                log = 'Iter: {:04d}|Train Loss: {:.4f}|Train MAPE: {:.4f}|Train RMSE: {:.4f}|Train MAE: {:.4f}' \
                      '|Time: {:.4f} '
                print(log.format(iter,train_loss[-1],train_mape[-1],train_rmse[-1],train_mae[-1],time.time()-t),
                      flush=True)
                t = time.time()
        scheduler.step()
        t2 = time.time()
        train_time.append(t2 - t1)

        #----------------------------------------------------------------------
        #------------------------------Validation------------------------------
        #----------------------------------------------------------------------
        valid_loss, valid_mape, valid_rmse, valid_mae = [], [], [], []
        s1 = time.time()
        for iter, (x, y) in enumerate(dataloader['val_loader'].get_iterator()):
            valX = torch.Tensor(x).to(device).transpose(1, 2)
            valY = torch.Tensor(y).to(device).transpose(1, 2)

            model.eval()
            y_pred = model(valX)
            y_pred = scaler.inverse_transform(y_pred)
            loss = criterion(y_pred, valY, 0.0)

            optimizer.zero_grad()
            evaluation = evaluate(y_pred, valY)
            valid_loss.append(loss.item())
            valid_mape.append(evaluation[0])
            valid_rmse.append(evaluation[1])
            valid_mae.append(evaluation[2])

        s2 = time.time()
        log = 'Epoch: {:03d}, Inference Time: {:.4f} secs'
        print(log.format(epoch+1, (s2 - s1)))
        val_time.append(s2 - s1)

        mtrain_loss = np.mean(train_loss)
        mtrain_mape = np.mean(train_mape)
        mtrain_rmse = np.mean(train_rmse)
        mtrain_mae = np.mean(train_mae)

        mvalid_loss = np.mean(valid_loss)
        mvalid_mape = np.mean(valid_mape)
        mvalid_rmse = np.mean(valid_rmse)
        mvalid_mae = np.mean(valid_mae)
        his_loss.append(mvalid_loss)

        message = dict(train_loss=mtrain_loss, train_mape=mtrain_mape, train_rmse=mtrain_rmse,
                        valid_loss=mvalid_loss, valid_mape=mvalid_mape, valid_rmse=mvalid_rmse)
        message = pd.Series(message)
        record.append(message)

        #----------------------------------------------------------------------
        #------------------------------Check point-----------------------------
        #----------------------------------------------------------------------
        if check_point_exist:
            checkpoint = torch.load(check_point_path)
            loss_check_pt = checkpoint['loss']
            if message.valid_loss < loss_check_pt:
                loss_check_pt = message.valid_loss
                torch.save({'epoch': epoch,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'loss': message.valid_loss,
                            }, check_point_path)
                epochs_since_best_mae = 0
                trigger_times = 0
            else:
                epochs_since_best_mae += 1
                trigger_times += 1
        else:
            torch.save({'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': message.valid_loss,
                        }, check_point_path)
            epochs_since_best_mae = 0
            trigger_times = 0
            check_point_exist = True


        #----------------------------------------------------------------------
        #------------------------------saving record---------------------------
        #----------------------------------------------------------------------
        record_df = pd.DataFrame(record)
        record_df.round(3).to_csv(record_path)

        #----------------------------------------------------------------------
        #---------------------------Displaying results-------------------------
        #----------------------------------------------------------------------
        log = 'Epoch: {:04d}, Training Time: {:.4f}/epoch,\n' \
              'Train Loss: {:.4f}, Train MAPE: {:.4f}, Train RMSE: {:.4f}, Train MAE: {:.4f}, \n' \
              'Valid Loss: {:.4f}, Valid MAPE: {:.4f}, Valid RMSE: {:.4f}, Valid MAE: {:.4f},'
        print(log.format(epoch+1, (t2 - t1),
                          mtrain_loss, mtrain_mape, mtrain_rmse, mtrain_mae,
                          mvalid_loss, mvalid_mape, mvalid_rmse, mvalid_mae), flush=True)
        print("*" * 100)

        #----------------------------------------------------------------------
        #---------------------------Early stopping-----------------------------
        #----------------------------------------------------------------------
        if trigger_times >= patience:
            print("Early stopping after {:03d} epochs!\nStart to test process.".format(epochs_since_best_mae))
            break

    #------------------------------------------------------------------------
    #-------------------------Displaying time--------------------------------
    #------------------------------------------------------------------------
    print("=" * 10)
    print("Average Train Time: {:.4f} secs/epoch".format(np.mean(train_time)))
    print("Average Valid Time: {:.4f} secs".format(np.mean(val_time)))
    print("=" * 10)


    #------------------------------------------------------------------------
    #------------------------------Testing-----------------------------------
    #------------------------------------------------------------------------
    best_epoch = np.argmin(his_loss)

    checkpoint = torch.load(check_point_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    test_mape, test_rmse, test_mae = [], [], []

    for iter, (x, y) in enumerate(dataloader['test_loader'].get_iterator()):
        testX = torch.Tensor(x).to(device).transpose(1, 2)
        testY = torch.Tensor(y).to(device).transpose(1, 2)

        with torch.no_grad():
            model.eval()
            y_pred = model(testX)
            y_pred = scaler.inverse_transform(y_pred)

        evaluation = evaluate(y_pred, testY)
        test_mape.append(evaluation[0])
        test_rmse.append(evaluation[1])
        test_mae.append(evaluation[2])

    log = 'Evaluate on test data : Test MAPE: {:.4f}, Test RMSE: {:.4f}, Test MAE: {:.4f}'
    print("=" * 10)
    print("Best epoch: ", best_epoch+1)
    print("checkpoint epoch: {:03d}  loss: {:.4f}".format(epoch, loss))
    print(log.format(np.mean(test_mape), np.mean(test_rmse), np.mean(test_mae)))
    print("=" * 10)


# Visualization

In [ ]:
#------------------------------------------------------------------------
#------------------------Visualization Functions-------------------------
#------------------------------------------------------------------------
import matplotlib.dates as mdates

def get_graph_path(seq_len=seq_len,
                   pre_len=pre_len,
                   data_name=data_name,
                   model_name=model_name):
    path_0 = '/content/drive/My Drive/Congestion/graph/%s/'%(model_name)
    path_1 = '%s_seq%r_pre%r_sensor%r'%(data_name,seq_len,pre_len,sensor)
    path = os.path.join(path_0,path_1)
    if not os.path.exists(path_0):
        os.makedirs(path_0)
    return path

def create_graph(y_true, y_pred, g_title, path_graph, ds_name, start_end):
    plt.rcParams['figure.figsize']=(12, 5)
    fig, ax = plt.subplots()
    g_ylim_all, g_ylim_one = [0, 80], [0, 80]
    loc='lower right'
    g_dpi = 600 #300, 600, 800

    plt.rc('xtick', labelsize=14)
    plt.rc('ytick', labelsize=14)

    a_true = y_true[start_end[0]:start_end[1]]
    a_pred = y_pred[start_end[0]:start_end[1]]

    path = '_start'+str(start_end[0])+'_end'+str(start_end[1])+'_'+g_title+'_'+str(g_dpi)+'.jpg'
    ax.set_ylim(g_ylim_all[0], g_ylim_all[1])

    ax.plot(a_true, 'b-', label='Ground Truth')
    ax.plot(a_pred, 'r-', label='STAT')
    ax.legend(loc=loc, ncol=2, facecolor='yellow', columnspacing=1.5, fontsize=14)

    ax.set_ylabel('Traffic Speed', fontweight='bold', fontsize=14)
    ax.set_xlabel('Time', fontweight='bold', fontsize=14)

    if(g_title == 'all'):
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

#    for label in ax.get_xticklabels(which='major'):
#        label.set(rotation=20, horizontalalignment='right')
    ax.grid(True)
    plt.savefig(path_graph+path, dpi=g_dpi)
    plt.show()


#------------------------------------------------------------------------
#------------------------------Visualization-----------------------------
#------------------------------------------------------------------------
run = 0
sensor = 1
one = [2528, 2816] # sf bay(10,20,30,40)[1740, 2028]/ tf bay(100,150)[3180, 3468]/ ltp la(0,1)[2528, 2816]
all = [2528, 5408] # sf bay(10,20,30,40)[1740, 4620]/ tf bay(100,150)[3180, 6060]/ ltp la(0,1)[2528, 5408]


best_model_path = '_la_final0079'

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Let's use {} GPU!".format(device))
else:
    device = torch.device("cpu")


trainX, trainY, valX, valY, testX, testY = get_data(seq_len, pre_len, data_name)
d_model = trainX.shape[-1]

dataloader = load_dataset(device, trainX, trainY, valX, valY, testX, testY, batch_size)
scaler = dataloader['scaler']

path_0 = '/content/drive/My Drive/Congestion/JA_model_pytorch/%s/'%(model_name)
path_2 = '%s/epoch%r/seq%r_pre%r_run%r_'%(data_name, n_epochs, seq_len, pre_len, run+1)
save_check_point = os.path.join(path_0,path_2)
mkdir(save_check_point)

model = STAT_model(device = device,
                    d_model = d_model,
                    n_layers_att = n_layers_att,
                    n_layers = n_layers,
                    FFN_units = FFN_units,
                    hid = hid,
                    dropout_rate = dropout
                    ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

check_point_path = save_check_point +'check_point_model'+best_model_path+'.pt'
checkpoint = torch.load(check_point_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

test_mape, test_rmse, test_mae = [], [], []

testX = torch.Tensor(testX).to(device).transpose(1, 2)
testY = torch.Tensor(testY).to(device).transpose(1, 2)

for iter, (x, y) in enumerate(dataloader['test_loader'].get_iterator()):
    testX = torch.Tensor(x).to(device).transpose(1, 2)
    testY = torch.Tensor(y).to(device).transpose(1, 2)

    with torch.no_grad():
        model.eval()
        predY = model(testX)
        predY = scaler.inverse_transform(predY)
    if (iter == 0):
        testY_1 = testY
        predY_1 = predY
    else:
        testY_1 = torch.cat((testY_1, testY), 0)
        predY_1 = torch.cat((predY_1, predY), 0)

y_true = testY_1[:, sensor, -1].to("cpu")
y_pred = predY_1[:, sensor, -1].to("cpu")
y_true = pd.DataFrame(y_true)
y_pred = pd.DataFrame(y_pred)
time_len, d_model = y_true.shape
rng = pd.date_range(end=test_date, periods=time_len, freq=freq)
y_true.index = rng
y_pred.index = rng
path_graph = get_graph_path(seq_len, pre_len, data_name, model_name)

create_graph(y_true, y_pred, 'one', path_graph, data_name, one)
create_graph(y_true, y_pred, 'all', path_graph, data_name, all)
